In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import random


rawdir = "content/drive/My Drive/valid/rawdata"
indir = "content/drive/My Drive/valid/inputdata"

filepath = ["air_visit_data.csv.zip"
            ,"date_info.csv.zip"]

In [ ]:
indir = "/content/drive/My Drive/valid/inputdata"

filepath = ["air_visit_data.csv",
            "air_store_info.csv"]

In [ ]:
import polars as pl
import os

visit_data = pl.read_csv(os.path.join(indir,filepath[0]))
info_data = pl.read_csv(os.path.join(indir,filepath[1]))

In [ ]:
visit_data.group_by("air_store_id").agg(pl.sum("visitors")).sort("visitors",descending =True)
# .sort("visitors")

In [ ]:
n_store_df=visit_data.filter(pl.col("air_store_id")=="air_399904bdb7685ca0")

In [ ]:
n_store_df.describe()

In [ ]:
filterd_df=info_data.filter(pl.col("air_store_id")=="air_399904bdb7685ca0")

In [ ]:
date_info = (
    pl.read_csv(os.path.join(indir,"date_info.csv"))
    .rename({"calendar_date": "visit_date"})
    # .rename(columns={"calendar_date": "visit_date"})
    .drop("day_of_week")
)
date_info 

In [ ]:
air_reserve = pl.read_csv(os.path.join(indir,"air_reserve.csv")).filter(pl.col("air_store_id")=="air_399904bdb7685ca0")
# air_reserve
# この辺りは特徴量を作成する前準備として、カラムを増やす処理を行っています
# 予約の日時から予約日を抽出します
air_reserve = air_reserve.with_columns(
    pl.col("visit_datetime").cast(pl.String).str.slice(0, 10).alias("visit_date"),
    pl.col("reserve_datetime").cast(pl.String).str.slice(0, 10).alias("reserve_date"),
    ).with_columns(
        pl.col("visit_date").str.to_date().dt.weekday().alias("dow")
      )
air_reserve
# joined_n_store_df

In [ ]:
n_store_df = n_store_df.with_columns(
    pl.format("{}_{}", pl.col("air_store_id"), pl.col("visit_date")).alias("id"),
    pl.col("visit_date").str.to_date().dt.weekday().alias("dow")
)
    # .str.to_date().dt.weekday().alias("dow")
n_store_df
# # 訪問者数のデータの店舗のIDと日付の情報からIDを作成します
# air_visit_max["id"] = air_visit_max["air_store_id"] + "_" + air_visit_max["visit_date"]
# # 訪問者数のデータの日付の情報から曜日を抽出します
# air_visit_max["dow"] = pd.to_datetime(air_visit_max["visit_date"]).dt.dayofweek

In [ ]:
date_info = date_info.with_columns(
    pl.col("visit_date").str.to_date().dt.weekday().alias("dow"),
    )

    # polars と　pandas　で　dt.weekdayの数値が変化する
date_info = date_info.with_columns(
    (
        (pl.col("dow") > 5) | (pl.col("holiday_flg") == 1)
    ).cast(pl.Int32).alias("holiday_flg2")
)

In [ ]:
joined_n_store_df = n_store_df.join(
    date_info[["visit_date", "holiday_flg", "holiday_flg2"]],
    on="visit_date",
    how="left",
)
joined_n_store_df.head(7)

In [ ]:

# 予測対象の訪問数に対数をとった場合の特徴量を作成する
train_log, test_log, predictors_log, target_log = feature_engineering(
    air_visit_max, air_reserve, date_info, is_log_transform=True
)
# 予測対象の訪問数から特徴量を作成する
train, test, predictors, target = feature_engineering(
    air_visit_max, air_reserve, date_info, is_log_transform=False
)

# LightGBMの学習する際のパラメーター
params = {
    "learning_rate": 0.02,
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "sub_feature": 0.7,
    "num_leaves": 60,
    "min_data": 1,
    "min_hessian": 1,
    "verbose_eval": -1,
}

# 訪問者数に対数をとった場合のデータセットを学習に使う特徴量と目的変数をLightGBM用のDatasetに格納する
lgb_train_log = lgb.Dataset(train_log[predictors_log], target_log)
# LightGBMを学習する
gbm_log = lgb.train(params, lgb_train_log, 1000)
# LightGBMのモデルを使って、テスト用の特徴量から訪問者数を予測
pred_log = gbm_log.predict(test_log[predictors_log])

# 訪問者数のデータセットを学習に使う特徴量と目的変数をLightGBM用のDatasetに格納する
lgb_train = lgb.Dataset(train[predictors], target)
# LightGBMを学習する
gbm = lgb.train(params, lgb_train, 1000)
# LightGBMのモデルを使って、テスト用の特徴量から訪問者数を予測
pred = gbm.predict(test[predictors])

# 教師データである訪問者数に対数をとった場合のデータセットを使って学習したモデルの訪問者数の予測精度を測るためにMean Absolute Errorを求める
mae_log = f"{mean_absolute_error(test['visitors'].values,np.expm1(pred_log)):.3f}"
# 訪問者数のデータセットを使って学習したモデルの訪問者数の予測精度を測るためにMean Absolute Errorを求める
mae = f"{mean_absolute_error(test['visitors'].values,pred):.3f}"

print(pd.DataFrame([[mae_log, mae]], columns=["モデル1", "モデル2"], index=["mae"]))